# Clustering Crypto

In [649]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [650]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('crypto_data.csv', index_col=0)

In [651]:
# Keep all the cryptocurrencies that are being traded.
crypto_df['IsTrading'] = crypto_df['IsTrading'].astype(str)
crypto_df['IsTrading'] = crypto_df['IsTrading'].str.contains("True")

In [652]:
# Keep all the cryptocurrencies that have a working algorithm.


In [653]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns = ['IsTrading'])

In [654]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [655]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [656]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name = crypto_df['CoinName']

In [657]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns = ['CoinName'])
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [658]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns = ['Algorithm', 'ProofType'])

In [659]:
# Standardize the data with StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [660]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [661]:
# Create a DataFrame with the three principal components.
X_pca = pca.fit_transform(X)
pcs_df = pd.DataFrame(data=X_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.334601,1.145390,-0.485163
404,-0.317277,1.145659,-0.485670
1337,2.415451,1.674753,-0.577051
BTC,-0.134890,-1.379568,0.170900
ETH,-0.140984,-2.157449,0.417917


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [662]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [663]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)
# Fit the model

model.fit(pcs_df)


# Predict clusters
predictions = model.predict(pcs_df)
    
pcs_df["class"] = model.labels_
    


In [664]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis = 1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['Name'] = crypto_name

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(577, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,Name
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334601,1.145390,-0.485163,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317277,1.145659,-0.485670,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.415451,1.674753,-0.577051,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.134890,-1.379568,0.170900,2,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.140984,-2.157449,0.417917,2,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.169639,-1.045282,-0.000312,2,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.372640,1.257966,-0.396791,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.130582,-2.275145,0.386520,2,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.139368,-2.157539,0.417888,2,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.145019,-2.045614,0.190105,2,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [665]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name="Name",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [666]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(sortable=True, selectable=True)


:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,Name]

In [667]:
# Print the total number of tradable cryptocurrencies.
print(len(clustered_df['Name']))

577


In [668]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
mms = MinMaxScaler()
clustered_df[['TotalCoinSupply','TotalCoinsMined']] = mms.fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])

In [670]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = clustered_df.filter(['TotalCoinSupply','TotalCoinsMined'])
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['Name']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df['class']
plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,2
ETH,0.000000e+00,0.000109,Ethereum,2
LTC,8.400000e-05,0.000064,Litecoin,2
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,2
ETC,2.100000e-04,0.000115,Ethereum Classic,2
ZEC,2.100000e-05,0.000007,ZCash,2


In [674]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols="CoinName" )

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)